# Exercício 1
Faça uma função `matrixPageRank(arquivo, alpha)` que recebe um digrafo $D$ no arquivo `arquivo` e um número $\alpha$ e devolve a matriz 
$$M = \alpha S + (1-\alpha) J$$.

O digrafo é dado no formato em https://snap.stanford.edu/data/p2p-Gnutella06.html em um arquivo txt.

Rode para o digrafo do link (mas não imprima a matriz inteira, o digrafo é grande) com $\alpha = 0.85$

**Definições**: Para um digrafo com $n$ vértices:
* $J$ é a matriz $n$ por $n$ em que todas as entradas são 1
* $S$ é a matriz obtida da matriz $H$ em que colunas de zeros são substituídas por colunas em que todas as entradas são $1/n$
* $H$ é a matriz $n$ por $n$ tal que
    * $H(i,j) = 1/links(j)$, se $j$ tem um link para $i$ e $links(j)$ é o número de links saindo de $j$;
    * $H(i,j) = 0$, caso contrário.
    
**Sugestão: use o numpy**

**Crie quantas células de código quiser**


In [1]:
import numpy as np

In [2]:
n = 8717

In [3]:
def matrixPageRank(arquivo, alpha):
    
    # Arquivo:
    # Directed graph (each unordered pair of nodes is saved once): p2p-Gnutella06.txt 
    # Directed Gnutella P2P network from August 6 2002
    # Nodes: 8717 Edges: 31525
    # FromNodeId	ToNodeId   

    edges_list = np.array(arquivo.readlines())
    
    print("Edges = ", len(edges_list))
    
    edges = np.zeros((31525,2),dtype=int)
    
    for i in range(31525):
                    
        edge = edges_list[i].split()
        edges[i,0] = edge[0]
        edges[i,1] = edge[1]
        
    J = np.ones((n,n))
    H = np.zeros((n,n))
    
    jlinks = np.zeros(n)
    
    #counting degree of each vextex
    for i in range(31525):
        jlinks[edges[i][0]] +=1
    
    #creating matrix H
    for s in range(31525):
        H[edges[s][1]][edges[s][0]] = 1/jlinks[edges[s][0]]

    
    #creating matrix S
    S = np.copy(H)
    for j in range(n):
        contzeros = 0
        for i in range(n):
            if S[i][j] == 0:
                contzeros += 1

        if contzeros == n:
            for s in range(n):
                S[s][j] = 1/n

        
    M = alpha*S + (1-alpha)*J
    return M

In [4]:
arquivo = open("p2p-Gnutella06.txt","r")
alpha = 0.85

M = matrixPageRank(arquivo, alpha)

arquivo.close()

Edges =  31525


In [5]:
print(M)

[[0.15       0.15009751 0.15009751 ... 0.15       0.15009751 0.15      ]
 [0.235      0.15009751 0.15009751 ... 0.15       0.15009751 0.15      ]
 [0.235      0.15009751 0.15009751 ... 0.15       0.15009751 0.15      ]
 ...
 [0.15       0.15009751 0.15009751 ... 0.15       0.15009751 0.15      ]
 [0.15       0.15009751 0.15009751 ... 0.15       0.15009751 0.15      ]
 [0.15       0.15009751 0.15009751 ... 0.235      0.15009751 0.15      ]]


# Exercício 2
Faça uma função `resolve_ex2(M)` que retorna o vetor de relevância $r$ para a matriz $M$. 

Para encontrar $r$, sugiro utilizar o pacote numpy.linalg com a função eig.

Calcule o tempo que demora para resolver o digrafo grande do link.

In [6]:
def resolve_ex2(M):
    
    start = time.time()
    
    vals, vecs = np.linalg.eig(M)
    
    end = time.time()

    elapsed = end - start
    
    print("Tempo de execucao: ", elapsed, "segundos")
    
    return np.abs(np.real(vecs[:n,0])/np.linalg.norm(vecs[:n,0],1))

In [7]:
import time
ave = resolve_ex2(M)
print("Autovetor: \n", ave)

Tempo de execucao:  544.591735124588 segundos
Autovetor: 
 [0.0001147  0.00011472 0.00011479 ... 0.00011469 0.0001147  0.00011469]


# Exercício 3
Faça uma função `resolve_ex3(M, k)` que retorna uma aproximação para o vetor de relevância $r$ utilizando o power method.

Lembre-se que o power method nos diz que para $M^k x$ converge para $r$ com $k\to\infty$ e $x = (1/n,\dotsc, 1/n)$.

Ou seja, basta calcular $M^k x$

Calcule o tempo que demora para resolver o digrafo grande do link (com $k=50$).

In [8]:
def resolve_ex3(M, k):
    
    start = time.time()
    
    i = 0
    Mk = np.copy(M)
    
    while i<k:
        
        Mk = np.dot(Mk,M)
        i += 1
        
    x = np.ones(n)
    
    x = x*1/n
    
    r = np.dot(Mk,x)
    
    end = time.time()
    
    elapsed = end - start
    print("Tempo de execucao: ",elapsed, "segundos")
    
    return r

In [9]:
r = resolve_ex3(M,50)
print("Vetor de Relevancia: \n",r)

Tempo de execucao:  2398.980457305908 segundos
Vetor de Relevancia: 
 [1.03118284e+155 1.03132424e+155 1.03199409e+155 ... 1.03111579e+155
 1.03116045e+155 1.03111579e+155]


# Exercício 4
Usando os exercícios anteriores, faça uma função `top20` que recebe M e r e imprime os 20 sites mais relevantes de acordo com $r$.

Compare essa lista para r obtido no Exercício 2 e no Exercício 3. Qual o menor $k$ para qual as listas de top 20 coincindem?

In [27]:
def top20(M,r):
    
    print("Comparando resultado com o metodo de autovetor e PowerMethod com k = 50")
    ordemr = np.sort(r)
    ordemave = np.sort(ave)
    indicer = []
    indiceave = []
    
    r1 = np.copy(r)
    ave1 = np.copy(ave)
    
    for i in range(20):
        for j in range(n):
            if ordemr[n-i-1] == r1[j]:
                indicer.append(j)
                r1[j] = 0
            
            if ordemave[n-i-1] == ave1[j]:
                indiceave.append(j)
                ave1[j] = 0
    
    for i in range(20):        
        print("PowerMethod: ", indicer[i], " EigenvectorMethod: ", indiceave[i])
                
    equal = 0 
    k = 0
    
    Mk = np.copy(M)
    
    while(equal != 20):
        
        equal = 0
        
        k += 1
     
        Mk = np.dot(Mk,M)    
        
        x = np.ones(n)
        
        x = x*1/n
        
        r2 = np.dot(Mk,x)      
        
        ordemr2 = np.sort(r2)
        
        indicer2 = []
        
        for i in range(20):
            for j in range(n):
                if ordemr2[n-i-1] == r2[j]:
                    indicer2.append(j)
                    r2[j] = 0
        
        for s in range(20):
            if indiceave[s] == indicer2[s]:
                equal += 1
                        
    print("\n\nIdentificando o menor k para o qual o resultado de ambos os metodos coincidem:")
    for i in range(20):        
        print("PowerMethod: ", indicer2[i], " EigenvectorMethod: ", indiceave[i])
    
    print("Minimo k = ", k)
    return indicer, indiceave

In [28]:
indicer, indiceave = top20(M, r)

Comparando resultado com o metodo de autovetor e PowerMethod com k = 50
PowerMethod:  1250  EigenvectorMethod:  1250
PowerMethod:  556  EigenvectorMethod:  556
PowerMethod:  356  EigenvectorMethod:  356
PowerMethod:  67  EigenvectorMethod:  67
PowerMethod:  176  EigenvectorMethod:  176
PowerMethod:  841  EigenvectorMethod:  841
PowerMethod:  31  EigenvectorMethod:  31
PowerMethod:  300  EigenvectorMethod:  300
PowerMethod:  299  EigenvectorMethod:  299
PowerMethod:  558  EigenvectorMethod:  558
PowerMethod:  293  EigenvectorMethod:  293
PowerMethod:  297  EigenvectorMethod:  297
PowerMethod:  173  EigenvectorMethod:  173
PowerMethod:  174  EigenvectorMethod:  174
PowerMethod:  353  EigenvectorMethod:  353
PowerMethod:  291  EigenvectorMethod:  291
PowerMethod:  461  EigenvectorMethod:  461
PowerMethod:  170  EigenvectorMethod:  170
PowerMethod:  99  EigenvectorMethod:  99
PowerMethod:  294  EigenvectorMethod:  294

 
 Identificando o menor k para o qual o resultado de ambos os metodos 

# Exercício 5
Faça uma função `naive(arquivo)` que recebe um digrafo (como no Exercício 1) e considera a relevância de uma página $p$ como o número de páginas apontando para $p$. Compare esse critério com o obtido no Exercício 4 para o digrafo grande.

In [29]:
def naive(arquivo):
    
    lista_de_edges = np.array(arquivo.readlines())
    
    print("Edges = ", len(lista_de_edges))
    
    edges = np.zeros((31525,2),dtype=int)
    
    for i in range(31525):
                    
        edge = lista_de_edges[i].split()
        edges[i,0] = edge[0]
        edges[i,1] = edge[1]
        
    #print("Arestas \n")
    #for i in range(31525):
    #    print(edges[i][0]," ", edges[i][1])
            
        
    rel = np.zeros(n, dtype=int)

    for i in range(31525):
        rel[edges[i][1]] +=1
    
    print("\n Relevancia \n")
    
    ordemrel = np.sort(rel)
    indicerel = []
    
    for i in range(20):
        for j in range(n):
            if ordemrel[n-i-1] == rel[j]:
                indicerel.append(j)
                rel[j] = 0
    
    for i in range(20):
        print("PowerMethod: ", indicer[i], " EigenvectorMethod: ", indiceave[i], "DegreeMethod : ", indicerel[i])

In [30]:
arquivo = open("p2p-Gnutella06.txt","r")
naive(arquivo)
arquivo.close()

Edges =  31525

 Relevancia 

PowerMethod:  1250  EigenvectorMethod:  1250 DegreeMethod :  356
PowerMethod:  556  EigenvectorMethod:  556 DegreeMethod :  176
PowerMethod:  356  EigenvectorMethod:  356 DegreeMethod :  31
PowerMethod:  67  EigenvectorMethod:  67 DegreeMethod :  556
PowerMethod:  176  EigenvectorMethod:  176 DegreeMethod :  293
PowerMethod:  841  EigenvectorMethod:  841 DegreeMethod :  299
PowerMethod:  31  EigenvectorMethod:  31 DegreeMethod :  558
PowerMethod:  300  EigenvectorMethod:  300 DegreeMethod :  173
PowerMethod:  299  EigenvectorMethod:  299 DegreeMethod :  300
PowerMethod:  558  EigenvectorMethod:  558 DegreeMethod :  67
PowerMethod:  293  EigenvectorMethod:  293 DegreeMethod :  174
PowerMethod:  297  EigenvectorMethod:  297 DegreeMethod :  291
PowerMethod:  173  EigenvectorMethod:  173 DegreeMethod :  297
PowerMethod:  174  EigenvectorMethod:  174 DegreeMethod :  99
PowerMethod:  353  EigenvectorMethod:  353 DegreeMethod :  170
PowerMethod:  291  Eigenvector

## Como podemos observar nenhuma relevancia no top20 obtida atraves do metodo do grau dos vertices e identica a relevancia obtida pelos outros dois metodos